In [ ]:
# ni tes dengan template
# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os

# Function to apply bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y_filtered = lfilter(b, a, data)
    return y_filtered


def remove_silence(y):
     # Memisahkan audio menjadi bagian-bagian berdasarkan energi
    parts = librosa.effects.split(y, top_db=25, frame_length=1024, hop_length=512)

    # Menggabungkan bagian-bagian yang tidak diam
    y_non_silent = []
    for start, end in parts:
        y_non_silent.extend(y[start:end])

    # Mengubah list menjadi array numpy
    y = np.array(y_non_silent)
    return y

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    # y = librosa.util.normalize(y)
    y = y/np.max(np.abs(y))
    y = remove_silence(y)
    y = nr.reduce_noise(y, sr, prop_decrease=0.5)
    # y = bandpass_filter(y, lowcut=100.0, highcut=3000.0, fs=sr, order=5)
    # y = shift_pitch_audio(y, sr)

    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=False,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm='ortho',
                                 dct_type=2,
                                 n_mels=20)
    # return = librosa.util.normalize(mfccs.T)
    return mfccs.T
def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(input_mfcc, template_mfcc, output_file, input_audio_name, template_score, max_threshold_score):
    final_score = 0

    with open(output_file, 'a') as f:  # Menggunakan mode 'a' untuk append
        # Hitung jarak antara input dan template menggunakan DTW
        distance = calculate_dtw(input_mfcc, template_mfcc)

        # Hitung selisih antara jarak dan nilai template
        dif_distance = distance - template_score

        # Normalisasi selisih antara 0 dan 1
        normalized_dif_distance = (dif_distance - 0) / (max_threshold_score - 0)

        # Hitung skor berdasarkan selisih yang dinormalisasi
        final_score = int(100 - (normalized_dif_distance * 100))

        # Pastikan skor berada dalam rentang 0-100
        final_score = max(min(final_score, 100), 0)

        # Tulis nama audio dan hasil jarak ke file
        f.write(f"Audio Name: {input_audio_name}\n")
        f.write(f"Distance = {distance}\n")
        # f.write(f"Difference Distance = {dif_distance}\n")
        # f.write(f"Final Score = {final_score}\n")
        f.write("\n")
        print(f"Audio Name: {input_audio_name}")
        print(f"Distance = {distance}\n")
        # print(f"Difference Distance = {dif_distance}\n")
        # print(f"final score = {final_score}\n")

# List of audio files and their names
list_of_audio_files = [
    ("../Dataset/Testing_Rafi/rafi_shad.wav", "rafi_shad"),
    ("../Dataset/Ridho_mic/Ridho_shad.wav", "ridho_shad"),
    ("../Dataset/Dona_mic/.wav", "ridho_shad")
]

# Output file path
# output_file = "../hasil_testing_template/newpre_testing_non_norm_template_shad.txt"
output_file = "../hasil_testing_template/testing_langsung_mfcc.txt"

# Load template MFCC
# template_mfcc_path = "../new_hasil_template_mfcc/newpre_non_norm_template_shad.csv"
template_mfcc_path = "../new_hasil_template_mfcc/shad_mfcc_1.csv"
template_mfcc = np.loadtxt(template_mfcc_path, delimiter=",")
# template_mfcc = librosa.util.normalize(template_mfcc)

# Threshold scores
template_score = 309.962
max_threshold_score = 150
with open(output_file, 'w') as f:
        f.write('')

# Write header to the output file
with open(output_file, 'w') as f:
    f.write("Hasil Banding Template new preprocessing:\n\n")

# Iterate over audio files
for input_file_audio, input_audio_name in list_of_audio_files:
    # Extract MFCC from current audio file
    input_mfcc = extraction(input_file_audio)

    # Call main function for current audio file
    main(input_mfcc, template_mfcc, output_file, input_audio_name, template_score, max_threshold_score)



In [4]:
# tes

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from scipy.signal import butter, lfilter
from IPython.display import clear_output
from sklearn.decomposition import PCA

def remove_silence(y):
    # Memisahkan audio menjadi bagian-bagian berdasarkan energi
    parts = librosa.effects.split(y, top_db=25, frame_length=1024, hop_length=512)

    # Menggabungkan bagian-bagian yang tidak diam
    y_non_silent = []
    for start, end in parts:
        y_non_silent.extend(y[start:end])

    # Mengubah list menjadi array numpy
    y = np.array(y_non_silent)
    return y

def preprocessing(audio):
    y, sr = librosa.load(audio, sr=44100)
    y = librosa.effects.preemphasis(y)
    y = y / np.max(np.abs(y))
    y = remove_silence(y)
    y = nr.reduce_noise(y, sr, prop_decrease=0.8)
    return y, sr

def extraction(audio):
    y, sr = preprocessing(audio)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=True,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm=None,
                                 dct_type=2,
                                 n_mels=20)
    return mfccs.T

def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, audio_name, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    total_distances = 0

    with open(output_file, 'a') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = np.loadtxt(audio_template, delimiter=',')

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            total_distances += dtw_distance

        f.write(audio_name)
        print(audio_name)
        f.write(f"\nTotal Distance = {total_distances}\n")
        print(f"\nTotal Distance = {total_distances}\n")

        average_distance = total_distances / len(templates)
        f.write(f"Average Distance = {average_distance}\n")
        print(f"Average Distance = {average_distance}\n")


root = "../Dataset_MFCC_95prem/"

list_template_folders = [
    ("01.Ha'/"),
    # ("02.Kha'/"),
    # ("03.Shad/"),
    # ("04.Dhad/"),
    # ("05.Tha'/"),
    # ("06.Dhza'/"),
    # ("07.'AIn/"),
    # ("08.Ghain/"),
    # ("09.Qaf/"),
    # ("10.Ha^'/")
]
# List of audio files and their names
list_of_audio_files = [
    ("../Dataset/Testing_Rafi/rafi_Ha'.wav", "rafi_Ha'"),
    ("../Dataset/Ridho_mic/Ridho_Ha'.wav", "ridho_Ha'"),
    ("../Dataset/Dona_mic/Ha'_F1_14.wav", "F1_Ha'"),
    ("../Dataset/Zahrah_mic/Ha'_F2_14.wav", "F2_Ha'")
]

for folder in list_template_folders:
    clear_output()
    template_folder = os.path.join(root, folder)

    output_file = f"../fix_hasil_testing/tes.txt"

    template_score = 23.004
    max_threshold_score = 5

    with open(output_file, 'w') as f:
        f.write('')
        f.write(f"Banding {folder.strip('/')} dengan tester\n")
    print(f"Processing Folder = {folder.strip('/')}")
    # Iterate over each audio file
    for audio_input, audio_name in list_of_audio_files:
        main(audio_input, audio_name, template_folder, output_file, template_score, max_threshold_score)


Processing Folder = 01.Ha'


AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [1]:
# ini tes banding mfcc per audio langsung dtw

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from scipy.signal import butter, lfilter
from IPython.display import clear_output
from sklearn.decomposition import PCA

def remove_silence(y):
     # Memisahkan audio menjadi bagian-bagian berdasarkan energi
    parts = librosa.effects.split(y, top_db=25, frame_length=1024, hop_length=512)

    # Menggabungkan bagian-bagian yang tidak diam
    y_non_silent = []
    for start, end in parts:
        y_non_silent.extend(y[start:end])

    # Mengubah list menjadi array numpy
    y = np.array(y_non_silent)
    return y

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y = librosa.effects.preemphasis(y)
    y = y/np.max(np.abs(y))
    y = remove_silence(y)
    y = nr.reduce_noise(y, sr, prop_decrease=0.8)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    # y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=True,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm=None,
                                 dct_type=2,
                                 n_mels=20)
    # mfccs = mfccs[1:]
    # mfccs = librosa.util.normalize(mfccs, axis=1 )
    # return = librosa.util.normalize(mfccs.T)
    return mfccs.T


def apply_pca(mfcc_features, n_components):
    pca = PCA(n_components=n_components)
    pca.fit(mfcc_features)
    mfcc_pca = pca.transform(mfcc_features)
    return mfcc_pca

def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, audio_name, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    # print(f"len templates = {len(templates)}")
    total_distances = 0

    with open(output_file, 'a') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = np.loadtxt(audio_template,delimiter=',')
            # mfccs2 = mfccs2.T
            # mfccs2 = mfccs2[1:]
            # mfccs2 = mfccs2.T

            # mfccs1 = apply_pca(mfccs1,10)
            # mfccs2 = apply_pca(mfccs2,10)
            # print(f"mfccs1.shape = {mfccs1.shape}")
            # f.write(f"mfccs1.shape = {mfccs1.shape}")
            # print(f"mfccs2.shape = {mfccs2.shape}")
            # f.write(f"mfccs2.shape = {mfccs2.shape}")

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            # print(f"dtw_distance = {dtw_distance}")
            # f.write(f"dtw_distance = {dtw_distance}")
            total_distances += dtw_distance

            # result_line = f"Jarak DTW antara {audio_input} dan {template}: {dtw_distance}\n"
            # f.write(result_line)

        # Tulis hasil total jarak ke file
        f.write(audio_name)
        print(audio_name)
        f.write(f"\nTotal Distance = {total_distances}\n")
        print(f"\nTotal Distance = {total_distances}\n")

        # Hitung rata-rata jarak
        average_distance = total_distances / len(templates)
        f.write(f"Average Distance = {average_distance}\n")
        print(f"Average Distance = {average_distance}\n")
        # # Hitung selisih antara rata-rata jarak dengan nilai template
        # dif_distance = average_distance - template_score
        # f.write(f"Difference Distance = {dif_distance}\n")

        # # Hitung skor akhir berdasarkan selisih
        # if dif_distance <= 0:
        #     final_score = 100
        # elif dif_distance >= max_threshold_score:
        #     final_score = 0
        # else:
        #     final_score = int(100 - ((dif_distance / max_threshold_score) * 100))
        
        # f.write(f"Final Score = {final_score}\n")




root = "../Dataset_MFCC_noprem/"

list_template_folders = [
    ("01.Ha'/"),
    ("02.Kha'/"),
    ("03.Shad/"),
    ("04.Dhad/"),
    ("05.Tha'/"),
    ("06.Dhza'/"),
    ("07.'AIn/"),
    ("08.Ghain/"),
    ("09.Qaf/"),
    ("10.Ha^'/")
]
# List of audio files and their names
list_of_audio_files = [
    ("../Dataset/Testing_Rafi/rafi_Ha'.wav", "rafi_Ha'"),
    ("../Dataset/Ridho_mic/Ridho_Ha'.wav", "ridho_Ha'"),
    ("../Dataset/Dona_mic/Ha'_F1_14.wav", "F1_Ha'"),
    ("../Dataset/Zahrah_mic/Ha'_F2_14.wav", "F2_Ha'")
]

for folder in list_template_folders:
    clear_output()
    template_folder = os.path.join(root, folder)

    output_file = f"../fix_hasil_testing/noprem_testing_Ha'_template_{folder.strip('/')}.txt"

    template_score = 23.004
    max_threshold_score = 5

    with open(output_file, 'w') as f:
            f.write('')
            f.write(f"Banding {folder.strip('/')} dengan tester\n")
    print(f"Processing Folder = {folder.strip('/')}")
    # Iterate over each audio file
    for audio_input, audio_name in list_of_audio_files:
        main(audio_input,audio_name, template_folder, output_file, template_score, max_threshold_score)
    


Processing Folder = 01.Ha'
rafi_Ha'

Total Distance = 355.5269541834126

Average Distance = 19.751497454634034

ridho_Ha'

Total Distance = 384.24146453766843

Average Distance = 21.34674802987047



KeyboardInterrupt: 

In [14]:
# ini tes banding mfcc per audio langsung dtw

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from scipy.signal import butter, lfilter
from IPython.display import clear_output
from sklearn.decomposition import PCA

def remove_silence(y):
     # Memisahkan audio menjadi bagian-bagian berdasarkan energi
    parts = librosa.effects.split(y, top_db=25, frame_length=1024, hop_length=512)

    # Menggabungkan bagian-bagian yang tidak diam
    y_non_silent = []
    for start, end in parts:
        y_non_silent.extend(y[start:end])

    # Mengubah list menjadi array numpy
    y = np.array(y_non_silent)
    return y

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y = librosa.effects.preemphasis(y)
    y = y/np.max(np.abs(y))
    y = remove_silence(y)
    y = nr.reduce_noise(y, sr, prop_decrease=0.8)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    # y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=True,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm=None,
                                 dct_type=2,
                                 n_mels=20)
    # mfccs = mfccs[1:]
    # mfccs = librosa.util.normalize(mfccs, axis=1 )
    # return = librosa.util.normalize(mfccs.T)
    return mfccs.T


def apply_pca(mfcc_features, n_components):
    pca = PCA(n_components=n_components)
    pca.fit(mfcc_features)
    mfcc_pca = pca.transform(mfcc_features)
    return mfcc_pca

def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, audio_name, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    # print(f"len templates = {len(templates)}")
    total_distances = 0

    with open(output_file, 'a') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = np.loadtxt(audio_template,delimiter=',')
            # mfccs2 = mfccs2.T
            # mfccs2 = mfccs2[1:]
            # mfccs2 = mfccs2.T

            # mfccs1 = apply_pca(mfccs1,10)
            # mfccs2 = apply_pca(mfccs2,10)
            # print(f"mfccs1.shape = {mfccs1.shape}")
            # f.write(f"mfccs1.shape = {mfccs1.shape}")
            # print(f"mfccs2.shape = {mfccs2.shape}")
            # f.write(f"mfccs2.shape = {mfccs2.shape}")

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            # print(f"dtw_distance = {dtw_distance}")
            # f.write(f"dtw_distance = {dtw_distance}")
            total_distances += dtw_distance

            # result_line = f"Jarak DTW antara {audio_input} dan {template}: {dtw_distance}\n"
            # f.write(result_line)

        # Tulis hasil total jarak ke file
        f.write(audio_name)
        print(audio_name)
        f.write(f"\nTotal Distance = {total_distances}\n")
        print(f"\nTotal Distance = {total_distances}\n")

        # Hitung rata-rata jarak
        average_distance = total_distances / len(templates)
        f.write(f"Average Distance = {average_distance}\n")
        print(f"Average Distance = {average_distance}\n")
        # # Hitung selisih antara rata-rata jarak dengan nilai template
        # dif_distance = average_distance - template_score
        # f.write(f"Difference Distance = {dif_distance}\n")

        # # Hitung skor akhir berdasarkan selisih
        # if dif_distance <= 0:
        #     final_score = 100
        # elif dif_distance >= max_threshold_score:
        #     final_score = 0
        # else:
        #     final_score = int(100 - ((dif_distance / max_threshold_score) * 100))
        
        # f.write(f"Final Score = {final_score}\n")




root = "../Dataset_MFCC_prem_norm_non_ortho/"

list_template_folders = [
    ("01.Ha'/"),
    ("02.Kha'/"),
    ("03.Shad/"),
    ("04.Dhad/"),
    ("05.Tha'/"),
    ("06.Dhza'/"),
    ("07.'AIn/"),
    ("08.Ghain/"),
    ("09.Qaf/"),
    ("10.Ha^'/")
]
# List of audio files and their names
list_of_audio_files = [
    ("../Dataset/Testing_Rafi/rafi_kha'.wav", "rafi_kha'"),
    ("../Dataset/Ridho_mic/Ridho_kha'.wav", "ridho_kha'"),
    ("../Dataset/Dona_mic/kha'_F1_14.wav", "F1_kha'"),
    ("../Dataset/Zahrah_mic/kha'_F2_14.wav", "F2_kha'")
]

for folder in list_template_folders:
    clear_output()
    template_folder = os.path.join(root, folder)

    output_file = f"../fix_hasil_testing/non_ortho_prem_norm_testing_kha'_template_{folder.strip('/')}.txt"

    template_score = 23.004
    max_threshold_score = 5

    with open(output_file, 'w') as f:
            f.write('')
            f.write(f"Banding {folder.strip('/')} dengan tester\n")
    print(f"Processing Folder = {folder.strip('/')}")
    # Iterate over each audio file
    for audio_input, audio_name in list_of_audio_files:
        main(audio_input,audio_name, template_folder, output_file, template_score, max_threshold_score)
    


Processing Folder = 10.Ha^'
rafi_kha'

Total Distance = 322.71305947316057

Average Distance = 17.928503304064478

ridho_kha'

Total Distance = 370.7166722496325

Average Distance = 20.59537068053514

F1_kha'

Total Distance = 302.3054599157834

Average Distance = 16.794747773099076

F2_kha'

Total Distance = 299.0364735225891

Average Distance = 16.613137417921617



In [15]:
# ini tes banding mfcc per audio langsung dtw

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from scipy.signal import butter, lfilter
from IPython.display import clear_output
from sklearn.decomposition import PCA

def remove_silence(y):
     # Memisahkan audio menjadi bagian-bagian berdasarkan energi
    parts = librosa.effects.split(y, top_db=25, frame_length=1024, hop_length=512)

    # Menggabungkan bagian-bagian yang tidak diam
    y_non_silent = []
    for start, end in parts:
        y_non_silent.extend(y[start:end])

    # Mengubah list menjadi array numpy
    y = np.array(y_non_silent)
    return y

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y = librosa.effects.preemphasis(y)
    y = y/np.max(np.abs(y))
    y = remove_silence(y)
    y = nr.reduce_noise(y, sr, prop_decrease=0.8)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    # y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=True,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm=None,
                                 dct_type=2,
                                 n_mels=20)
    # mfccs = mfccs[1:]
    # mfccs = librosa.util.normalize(mfccs, axis=1 )
    # return = librosa.util.normalize(mfccs.T)
    return mfccs.T


def apply_pca(mfcc_features, n_components):
    pca = PCA(n_components=n_components)
    pca.fit(mfcc_features)
    mfcc_pca = pca.transform(mfcc_features)
    return mfcc_pca

def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, audio_name, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    # print(f"len templates = {len(templates)}")
    total_distances = 0

    with open(output_file, 'a') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = np.loadtxt(audio_template,delimiter=',')
            # mfccs2 = mfccs2.T
            # mfccs2 = mfccs2[1:]
            # mfccs2 = mfccs2.T

            # mfccs1 = apply_pca(mfccs1,10)
            # mfccs2 = apply_pca(mfccs2,10)
            # print(f"mfccs1.shape = {mfccs1.shape}")
            # f.write(f"mfccs1.shape = {mfccs1.shape}")
            # print(f"mfccs2.shape = {mfccs2.shape}")
            # f.write(f"mfccs2.shape = {mfccs2.shape}")

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            # print(f"dtw_distance = {dtw_distance}")
            # f.write(f"dtw_distance = {dtw_distance}")
            total_distances += dtw_distance

            # result_line = f"Jarak DTW antara {audio_input} dan {template}: {dtw_distance}\n"
            # f.write(result_line)

        # Tulis hasil total jarak ke file
        f.write(audio_name)
        print(audio_name)
        f.write(f"\nTotal Distance = {total_distances}\n")
        print(f"\nTotal Distance = {total_distances}\n")

        # Hitung rata-rata jarak
        average_distance = total_distances / len(templates)
        f.write(f"Average Distance = {average_distance}\n")
        print(f"Average Distance = {average_distance}\n")
        # # Hitung selisih antara rata-rata jarak dengan nilai template
        # dif_distance = average_distance - template_score
        # f.write(f"Difference Distance = {dif_distance}\n")

        # # Hitung skor akhir berdasarkan selisih
        # if dif_distance <= 0:
        #     final_score = 100
        # elif dif_distance >= max_threshold_score:
        #     final_score = 0
        # else:
        #     final_score = int(100 - ((dif_distance / max_threshold_score) * 100))
        
        # f.write(f"Final Score = {final_score}\n")




root = "../Dataset_MFCC_prem_norm_non_ortho/"

list_template_folders = [
    ("01.Ha'/"),
    ("02.Kha'/"),
    ("03.Shad/"),
    ("04.Dhad/"),
    ("05.Tha'/"),
    ("06.Dhza'/"),
    ("07.'AIn/"),
    ("08.Ghain/"),
    ("09.Qaf/"),
    ("10.Ha^'/")
]
# List of audio files and their names
list_of_audio_files = [
    ("../Dataset/Testing_Rafi/rafi_shad.wav", "rafi_shad"),
    ("../Dataset/Ridho_mic/Ridho_shad.wav", "ridho_shad"),
    ("../Dataset/Dona_mic/shad_F1_14.wav", "F1_shad"),
    ("../Dataset/Zahrah_mic/shad_F2_14.wav", "F2_shad")
]

for folder in list_template_folders:
    clear_output()
    template_folder = os.path.join(root, folder)

    output_file = f"../fix_hasil_testing/non_ortho_prem_norm_testing_shad_template_{folder.strip('/')}.txt"

    template_score = 23.004
    max_threshold_score = 5

    with open(output_file, 'w') as f:
            f.write('')
            f.write(f"Banding {folder.strip('/')} dengan tester\n")
    print(f"Processing Folder = {folder.strip('/')}")
    # Iterate over each audio file
    for audio_input, audio_name in list_of_audio_files:
        main(audio_input,audio_name, template_folder, output_file, template_score, max_threshold_score)
    


Processing Folder = 10.Ha^'
rafi_shad

Total Distance = 648.8122227226581

Average Distance = 36.04512348459212

ridho_shad

Total Distance = 457.86157894372934

Average Distance = 25.43675438576274

F1_shad

Total Distance = 541.7978273542909

Average Distance = 30.099879297460607

F2_shad

Total Distance = 721.2159016106498

Average Distance = 40.06755008948054



In [16]:
# ini tes banding mfcc per audio langsung dtw

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from scipy.signal import butter, lfilter
from IPython.display import clear_output
from sklearn.decomposition import PCA

def remove_silence(y):
     # Memisahkan audio menjadi bagian-bagian berdasarkan energi
    parts = librosa.effects.split(y, top_db=25, frame_length=1024, hop_length=512)

    # Menggabungkan bagian-bagian yang tidak diam
    y_non_silent = []
    for start, end in parts:
        y_non_silent.extend(y[start:end])

    # Mengubah list menjadi array numpy
    y = np.array(y_non_silent)
    return y

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y = librosa.effects.preemphasis(y)
    y = y/np.max(np.abs(y))
    y = remove_silence(y)
    y = nr.reduce_noise(y, sr, prop_decrease=0.8)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    # y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=True,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm=None,
                                 dct_type=2,
                                 n_mels=20)
    # mfccs = mfccs[1:]
    # mfccs = librosa.util.normalize(mfccs, axis=1 )
    # return = librosa.util.normalize(mfccs.T)
    return mfccs.T


def apply_pca(mfcc_features, n_components):
    pca = PCA(n_components=n_components)
    pca.fit(mfcc_features)
    mfcc_pca = pca.transform(mfcc_features)
    return mfcc_pca

def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, audio_name, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    # print(f"len templates = {len(templates)}")
    total_distances = 0

    with open(output_file, 'a') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = np.loadtxt(audio_template,delimiter=',')
            # mfccs2 = mfccs2.T
            # mfccs2 = mfccs2[1:]
            # mfccs2 = mfccs2.T

            # mfccs1 = apply_pca(mfccs1,10)
            # mfccs2 = apply_pca(mfccs2,10)
            # print(f"mfccs1.shape = {mfccs1.shape}")
            # f.write(f"mfccs1.shape = {mfccs1.shape}")
            # print(f"mfccs2.shape = {mfccs2.shape}")
            # f.write(f"mfccs2.shape = {mfccs2.shape}")

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            # print(f"dtw_distance = {dtw_distance}")
            # f.write(f"dtw_distance = {dtw_distance}")
            total_distances += dtw_distance

            # result_line = f"Jarak DTW antara {audio_input} dan {template}: {dtw_distance}\n"
            # f.write(result_line)

        # Tulis hasil total jarak ke file
        f.write(audio_name)
        print(audio_name)
        f.write(f"\nTotal Distance = {total_distances}\n")
        print(f"\nTotal Distance = {total_distances}\n")

        # Hitung rata-rata jarak
        average_distance = total_distances / len(templates)
        f.write(f"Average Distance = {average_distance}\n")
        print(f"Average Distance = {average_distance}\n")
        # # Hitung selisih antara rata-rata jarak dengan nilai template
        # dif_distance = average_distance - template_score
        # f.write(f"Difference Distance = {dif_distance}\n")

        # # Hitung skor akhir berdasarkan selisih
        # if dif_distance <= 0:
        #     final_score = 100
        # elif dif_distance >= max_threshold_score:
        #     final_score = 0
        # else:
        #     final_score = int(100 - ((dif_distance / max_threshold_score) * 100))
        
        # f.write(f"Final Score = {final_score}\n")




root = "../Dataset_MFCC_prem_norm_non_ortho/"

list_template_folders = [
    ("01.Ha'/"),
    ("02.Kha'/"),
    ("03.Shad/"),
    ("04.Dhad/"),
    ("05.Tha'/"),
    ("06.Dhza'/"),
    ("07.'AIn/"),
    ("08.Ghain/"),
    ("09.Qaf/"),
    ("10.Ha^'/")
]
# List of audio files and their names
list_of_audio_files = [
    ("../Dataset/Testing_Rafi/rafi_dhad.wav", "rafi_dhad"),
    ("../Dataset/Ridho_mic/Ridho_dhad.wav", "ridho_dhad"),
    ("../Dataset/Dona_mic/dhad_F1_14.wav", "F1_dhad"),
    ("../Dataset/Zahrah_mic/dhad_F2_14.wav", "F2_dhad")
]

for folder in list_template_folders:
    clear_output()
    template_folder = os.path.join(root, folder)

    output_file = f"../fix_hasil_testing/non_ortho_prem_norm_testing_dhad_template_{folder.strip('/')}.txt"

    template_score = 23.004
    max_threshold_score = 5

    with open(output_file, 'w') as f:
            f.write('')
            f.write(f"Banding {folder.strip('/')} dengan tester\n")
    print(f"Processing Folder = {folder.strip('/')}")
    # Iterate over each audio file
    for audio_input, audio_name in list_of_audio_files:
        main(audio_input,audio_name, template_folder, output_file, template_score, max_threshold_score)
    


Processing Folder = 10.Ha^'
rafi_dhad

Total Distance = 330.3983736772654

Average Distance = 18.35546520429252

ridho_dhad

Total Distance = 318.398960947006

Average Distance = 17.688831163722554

F1_dhad

Total Distance = 366.90270258861926

Average Distance = 20.383483477145514

F2_dhad

Total Distance = 253.8695087616106

Average Distance = 14.103861597867256



In [17]:
# ini tes banding mfcc per audio langsung dtw

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from scipy.signal import butter, lfilter
from IPython.display import clear_output
from sklearn.decomposition import PCA

def remove_silence(y):
     # Memisahkan audio menjadi bagian-bagian berdasarkan energi
    parts = librosa.effects.split(y, top_db=25, frame_length=1024, hop_length=512)

    # Menggabungkan bagian-bagian yang tidak diam
    y_non_silent = []
    for start, end in parts:
        y_non_silent.extend(y[start:end])

    # Mengubah list menjadi array numpy
    y = np.array(y_non_silent)
    return y

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y = librosa.effects.preemphasis(y)
    y = y/np.max(np.abs(y))
    y = remove_silence(y)
    y = nr.reduce_noise(y, sr, prop_decrease=0.8)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    # y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=True,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm=None,
                                 dct_type=2,
                                 n_mels=20)
    # mfccs = mfccs[1:]
    # mfccs = librosa.util.normalize(mfccs, axis=1 )
    # return = librosa.util.normalize(mfccs.T)
    return mfccs.T


def apply_pca(mfcc_features, n_components):
    pca = PCA(n_components=n_components)
    pca.fit(mfcc_features)
    mfcc_pca = pca.transform(mfcc_features)
    return mfcc_pca

def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, audio_name, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    # print(f"len templates = {len(templates)}")
    total_distances = 0

    with open(output_file, 'a') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = np.loadtxt(audio_template,delimiter=',')
            # mfccs2 = mfccs2.T
            # mfccs2 = mfccs2[1:]
            # mfccs2 = mfccs2.T

            # mfccs1 = apply_pca(mfccs1,10)
            # mfccs2 = apply_pca(mfccs2,10)
            # print(f"mfccs1.shape = {mfccs1.shape}")
            # f.write(f"mfccs1.shape = {mfccs1.shape}")
            # print(f"mfccs2.shape = {mfccs2.shape}")
            # f.write(f"mfccs2.shape = {mfccs2.shape}")

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            # print(f"dtw_distance = {dtw_distance}")
            # f.write(f"dtw_distance = {dtw_distance}")
            total_distances += dtw_distance

            # result_line = f"Jarak DTW antara {audio_input} dan {template}: {dtw_distance}\n"
            # f.write(result_line)

        # Tulis hasil total jarak ke file
        f.write(audio_name)
        print(audio_name)
        f.write(f"\nTotal Distance = {total_distances}\n")
        print(f"\nTotal Distance = {total_distances}\n")

        # Hitung rata-rata jarak
        average_distance = total_distances / len(templates)
        f.write(f"Average Distance = {average_distance}\n")
        print(f"Average Distance = {average_distance}\n")
        # # Hitung selisih antara rata-rata jarak dengan nilai template
        # dif_distance = average_distance - template_score
        # f.write(f"Difference Distance = {dif_distance}\n")

        # # Hitung skor akhir berdasarkan selisih
        # if dif_distance <= 0:
        #     final_score = 100
        # elif dif_distance >= max_threshold_score:
        #     final_score = 0
        # else:
        #     final_score = int(100 - ((dif_distance / max_threshold_score) * 100))
        
        # f.write(f"Final Score = {final_score}\n")




root = "../Dataset_MFCC_prem_norm_non_ortho/"

list_template_folders = [
    ("01.Ha'/"),
    ("02.Kha'/"),
    ("03.Shad/"),
    ("04.Dhad/"),
    ("05.Tha'/"),
    ("06.Dhza'/"),
    ("07.'AIn/"),
    ("08.Ghain/"),
    ("09.Qaf/"),
    ("10.Ha^'/")
]
# List of audio files and their names
list_of_audio_files = [
    ("../Dataset/Testing_Rafi/rafi_tha'.wav", "rafi_tha'"),
    ("../Dataset/Ridho_mic/Ridho_tha'.wav", "ridho_tha'"),
    ("../Dataset/Dona_mic/tha'_F1_14.wav", "F1_tha'"),
    ("../Dataset/Zahrah_mic/tha'_F2_14.wav", "F2_tha'")
]

for folder in list_template_folders:
    clear_output()
    template_folder = os.path.join(root, folder)

    output_file = f"../fix_hasil_testing/non_ortho_prem_norm_testing_tha'_template_{folder.strip('/')}.txt"

    template_score = 23.004
    max_threshold_score = 5

    with open(output_file, 'w') as f:
            f.write('')
            f.write(f"Banding {folder.strip('/')} dengan tester\n")
    print(f"Processing Folder = {folder.strip('/')}")
    # Iterate over each audio file
    for audio_input, audio_name in list_of_audio_files:
        main(audio_input,audio_name, template_folder, output_file, template_score, max_threshold_score)
    


Processing Folder = 10.Ha^'
rafi_tha'

Total Distance = 264.2266583929609

Average Distance = 14.67925879960894

ridho_tha'

Total Distance = 321.6671801352177

Average Distance = 17.87039889640098

F1_tha'

Total Distance = 413.8758160321844

Average Distance = 22.99310089067691

F2_tha'

Total Distance = 279.1000952658345

Average Distance = 15.505560848101917



In [18]:
# ini tes banding mfcc per audio langsung dtw

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from scipy.signal import butter, lfilter
from IPython.display import clear_output
from sklearn.decomposition import PCA

def remove_silence(y):
     # Memisahkan audio menjadi bagian-bagian berdasarkan energi
    parts = librosa.effects.split(y, top_db=25, frame_length=1024, hop_length=512)

    # Menggabungkan bagian-bagian yang tidak diam
    y_non_silent = []
    for start, end in parts:
        y_non_silent.extend(y[start:end])

    # Mengubah list menjadi array numpy
    y = np.array(y_non_silent)
    return y

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y = librosa.effects.preemphasis(y)
    y = y/np.max(np.abs(y))
    y = remove_silence(y)
    y = nr.reduce_noise(y, sr, prop_decrease=0.8)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    # y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=True,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm=None,
                                 dct_type=2,
                                 n_mels=20)
    # mfccs = mfccs[1:]
    # mfccs = librosa.util.normalize(mfccs, axis=1 )
    # return = librosa.util.normalize(mfccs.T)
    return mfccs.T


def apply_pca(mfcc_features, n_components):
    pca = PCA(n_components=n_components)
    pca.fit(mfcc_features)
    mfcc_pca = pca.transform(mfcc_features)
    return mfcc_pca

def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, audio_name, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    # print(f"len templates = {len(templates)}")
    total_distances = 0

    with open(output_file, 'a') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = np.loadtxt(audio_template,delimiter=',')
            # mfccs2 = mfccs2.T
            # mfccs2 = mfccs2[1:]
            # mfccs2 = mfccs2.T

            # mfccs1 = apply_pca(mfccs1,10)
            # mfccs2 = apply_pca(mfccs2,10)
            # print(f"mfccs1.shape = {mfccs1.shape}")
            # f.write(f"mfccs1.shape = {mfccs1.shape}")
            # print(f"mfccs2.shape = {mfccs2.shape}")
            # f.write(f"mfccs2.shape = {mfccs2.shape}")

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            # print(f"dtw_distance = {dtw_distance}")
            # f.write(f"dtw_distance = {dtw_distance}")
            total_distances += dtw_distance

            # result_line = f"Jarak DTW antara {audio_input} dan {template}: {dtw_distance}\n"
            # f.write(result_line)

        # Tulis hasil total jarak ke file
        f.write(audio_name)
        print(audio_name)
        f.write(f"\nTotal Distance = {total_distances}\n")
        print(f"\nTotal Distance = {total_distances}\n")

        # Hitung rata-rata jarak
        average_distance = total_distances / len(templates)
        f.write(f"Average Distance = {average_distance}\n")
        print(f"Average Distance = {average_distance}\n")
        # # Hitung selisih antara rata-rata jarak dengan nilai template
        # dif_distance = average_distance - template_score
        # f.write(f"Difference Distance = {dif_distance}\n")

        # # Hitung skor akhir berdasarkan selisih
        # if dif_distance <= 0:
        #     final_score = 100
        # elif dif_distance >= max_threshold_score:
        #     final_score = 0
        # else:
        #     final_score = int(100 - ((dif_distance / max_threshold_score) * 100))
        
        # f.write(f"Final Score = {final_score}\n")




root = "../Dataset_MFCC_prem_norm_non_ortho/"

list_template_folders = [
    ("01.Ha'/"),
    ("02.Kha'/"),
    ("03.Shad/"),
    ("04.Dhad/"),
    ("05.Tha'/"),
    ("06.Dhza'/"),
    ("07.'AIn/"),
    ("08.Ghain/"),
    ("09.Qaf/"),
    ("10.Ha^'/")
]
# List of audio files and their names
list_of_audio_files = [
    ("../Dataset/Testing_Rafi/rafi_dhza'.wav", "rafi_dhza'"),
    ("../Dataset/Ridho_mic/Ridho_dhza'.wav", "ridho_dhza'"),
    ("../Dataset/Dona_mic/dhza'_F1_14.wav", "F1_dhza'"),
    ("../Dataset/Zahrah_mic/dhza'_F2_14.wav", "F2_dhza'")
]

for folder in list_template_folders:
    clear_output()
    template_folder = os.path.join(root, folder)

    output_file = f"../fix_hasil_testing/non_ortho_prem_norm_testing_dhza'_template_{folder.strip('/')}.txt"

    template_score = 23.004
    max_threshold_score = 5

    with open(output_file, 'w') as f:
            f.write('')
            f.write(f"Banding {folder.strip('/')} dengan tester\n")
    print(f"Processing Folder = {folder.strip('/')}")
    # Iterate over each audio file
    for audio_input, audio_name in list_of_audio_files:
        main(audio_input,audio_name, template_folder, output_file, template_score, max_threshold_score)
    


Processing Folder = 10.Ha^'
rafi_dhza'

Total Distance = 363.94173330842995

Average Distance = 20.218985183801664

ridho_dhza'

Total Distance = 352.2032064107973

Average Distance = 19.56684480059985

F1_dhza'

Total Distance = 512.1486680257626

Average Distance = 28.452703779209035

F2_dhza'

Total Distance = 278.4448606093621

Average Distance = 15.469158922742338



In [19]:
# ini tes banding mfcc per audio langsung dtw

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from scipy.signal import butter, lfilter
from IPython.display import clear_output
from sklearn.decomposition import PCA

def remove_silence(y):
     # Memisahkan audio menjadi bagian-bagian berdasarkan energi
    parts = librosa.effects.split(y, top_db=25, frame_length=1024, hop_length=512)

    # Menggabungkan bagian-bagian yang tidak diam
    y_non_silent = []
    for start, end in parts:
        y_non_silent.extend(y[start:end])

    # Mengubah list menjadi array numpy
    y = np.array(y_non_silent)
    return y

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y = librosa.effects.preemphasis(y)
    y = y/np.max(np.abs(y))
    y = remove_silence(y)
    y = nr.reduce_noise(y, sr, prop_decrease=0.8)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    # y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=True,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm=None,
                                 dct_type=2,
                                 n_mels=20)
    # mfccs = mfccs[1:]
    # mfccs = librosa.util.normalize(mfccs, axis=1 )
    # return = librosa.util.normalize(mfccs.T)
    return mfccs.T


def apply_pca(mfcc_features, n_components):
    pca = PCA(n_components=n_components)
    pca.fit(mfcc_features)
    mfcc_pca = pca.transform(mfcc_features)
    return mfcc_pca

def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, audio_name, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    # print(f"len templates = {len(templates)}")
    total_distances = 0

    with open(output_file, 'a') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = np.loadtxt(audio_template,delimiter=',')
            # mfccs2 = mfccs2.T
            # mfccs2 = mfccs2[1:]
            # mfccs2 = mfccs2.T

            # mfccs1 = apply_pca(mfccs1,10)
            # mfccs2 = apply_pca(mfccs2,10)
            # print(f"mfccs1.shape = {mfccs1.shape}")
            # f.write(f"mfccs1.shape = {mfccs1.shape}")
            # print(f"mfccs2.shape = {mfccs2.shape}")
            # f.write(f"mfccs2.shape = {mfccs2.shape}")

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            # print(f"dtw_distance = {dtw_distance}")
            # f.write(f"dtw_distance = {dtw_distance}")
            total_distances += dtw_distance

            # result_line = f"Jarak DTW antara {audio_input} dan {template}: {dtw_distance}\n"
            # f.write(result_line)

        # Tulis hasil total jarak ke file
        f.write(audio_name)
        print(audio_name)
        f.write(f"\nTotal Distance = {total_distances}\n")
        print(f"\nTotal Distance = {total_distances}\n")

        # Hitung rata-rata jarak
        average_distance = total_distances / len(templates)
        f.write(f"Average Distance = {average_distance}\n")
        print(f"Average Distance = {average_distance}\n")
        # # Hitung selisih antara rata-rata jarak dengan nilai template
        # dif_distance = average_distance - template_score
        # f.write(f"Difference Distance = {dif_distance}\n")

        # # Hitung skor akhir berdasarkan selisih
        # if dif_distance <= 0:
        #     final_score = 100
        # elif dif_distance >= max_threshold_score:
        #     final_score = 0
        # else:
        #     final_score = int(100 - ((dif_distance / max_threshold_score) * 100))
        
        # f.write(f"Final Score = {final_score}\n")




root = "../Dataset_MFCC_prem_norm_non_ortho/"

list_template_folders = [
    ("01.Ha'/"),
    ("02.Kha'/"),
    ("03.Shad/"),
    ("04.Dhad/"),
    ("05.Tha'/"),
    ("06.Dhza'/"),
    ("07.'AIn/"),
    ("08.Ghain/"),
    ("09.Qaf/"),
    ("10.Ha^'/")
]
# List of audio files and their names
list_of_audio_files = [
    ("../Dataset/Testing_Rafi/rafi_'ain.wav", "rafi_'ain"),
    ("../Dataset/Ridho_mic/Ridho_'ain.wav", "ridho_'ain"),
    ("../Dataset/Dona_mic/'ain_F1_14.wav", "F1_'ain"),
    ("../Dataset/Zahrah_mic/'ain_F2_14.wav", "F2_'ain")
]

for folder in list_template_folders:
    clear_output()
    template_folder = os.path.join(root, folder)

    output_file = f"../fix_hasil_testing/non_ortho_prem_norm_testing_'ain_template_{folder.strip('/')}.txt"

    template_score = 23.004
    max_threshold_score = 5

    with open(output_file, 'w') as f:
            f.write('')
            f.write(f"Banding {folder.strip('/')} dengan tester\n")
    print(f"Processing Folder = {folder.strip('/')}")
    # Iterate over each audio file
    for audio_input, audio_name in list_of_audio_files:
        main(audio_input,audio_name, template_folder, output_file, template_score, max_threshold_score)
    


Processing Folder = 10.Ha^'
rafi_'ain

Total Distance = 309.93059084993496

Average Distance = 17.21836615832972

ridho_'ain

Total Distance = 318.12970288210204

Average Distance = 17.673872382339002

F1_'ain

Total Distance = 417.31655985735694

Average Distance = 23.18425332540872

F2_'ain

Total Distance = 316.8026098284907

Average Distance = 17.600144990471705



In [20]:
# ini tes banding mfcc per audio langsung dtw

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from scipy.signal import butter, lfilter
from IPython.display import clear_output
from sklearn.decomposition import PCA

def remove_silence(y):
     # Memisahkan audio menjadi bagian-bagian berdasarkan energi
    parts = librosa.effects.split(y, top_db=25, frame_length=1024, hop_length=512)

    # Menggabungkan bagian-bagian yang tidak diam
    y_non_silent = []
    for start, end in parts:
        y_non_silent.extend(y[start:end])

    # Mengubah list menjadi array numpy
    y = np.array(y_non_silent)
    return y

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y = librosa.effects.preemphasis(y)
    y = y/np.max(np.abs(y))
    y = remove_silence(y)
    y = nr.reduce_noise(y, sr, prop_decrease=0.8)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    # y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=True,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm=None,
                                 dct_type=2,
                                 n_mels=20)
    # mfccs = mfccs[1:]
    # mfccs = librosa.util.normalize(mfccs, axis=1 )
    # return = librosa.util.normalize(mfccs.T)
    return mfccs.T


def apply_pca(mfcc_features, n_components):
    pca = PCA(n_components=n_components)
    pca.fit(mfcc_features)
    mfcc_pca = pca.transform(mfcc_features)
    return mfcc_pca

def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, audio_name, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    # print(f"len templates = {len(templates)}")
    total_distances = 0

    with open(output_file, 'a') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = np.loadtxt(audio_template,delimiter=',')
            # mfccs2 = mfccs2.T
            # mfccs2 = mfccs2[1:]
            # mfccs2 = mfccs2.T

            # mfccs1 = apply_pca(mfccs1,10)
            # mfccs2 = apply_pca(mfccs2,10)
            # print(f"mfccs1.shape = {mfccs1.shape}")
            # f.write(f"mfccs1.shape = {mfccs1.shape}")
            # print(f"mfccs2.shape = {mfccs2.shape}")
            # f.write(f"mfccs2.shape = {mfccs2.shape}")

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            # print(f"dtw_distance = {dtw_distance}")
            # f.write(f"dtw_distance = {dtw_distance}")
            total_distances += dtw_distance

            # result_line = f"Jarak DTW antara {audio_input} dan {template}: {dtw_distance}\n"
            # f.write(result_line)

        # Tulis hasil total jarak ke file
        f.write(audio_name)
        print(audio_name)
        f.write(f"\nTotal Distance = {total_distances}\n")
        print(f"\nTotal Distance = {total_distances}\n")

        # Hitung rata-rata jarak
        average_distance = total_distances / len(templates)
        f.write(f"Average Distance = {average_distance}\n")
        print(f"Average Distance = {average_distance}\n")
        # # Hitung selisih antara rata-rata jarak dengan nilai template
        # dif_distance = average_distance - template_score
        # f.write(f"Difference Distance = {dif_distance}\n")

        # # Hitung skor akhir berdasarkan selisih
        # if dif_distance <= 0:
        #     final_score = 100
        # elif dif_distance >= max_threshold_score:
        #     final_score = 0
        # else:
        #     final_score = int(100 - ((dif_distance / max_threshold_score) * 100))
        
        # f.write(f"Final Score = {final_score}\n")




root = "../Dataset_MFCC_prem_norm_non_ortho/"

list_template_folders = [
    ("01.Ha'/"),
    ("02.Kha'/"),
    ("03.Shad/"),
    ("04.Dhad/"),
    ("05.Tha'/"),
    ("06.Dhza'/"),
    ("07.'AIn/"),
    ("08.Ghain/"),
    ("09.Qaf/"),
    ("10.Ha^'/")
]
# List of audio files and their names
list_of_audio_files = [
    ("../Dataset/Testing_Rafi/rafi_Ghain.wav", "rafi_Ghain"),
    ("../Dataset/Ridho_mic/Ridho_Ghain.wav", "ridho_Ghain"),
    ("../Dataset/Dona_mic/Ghain_F1_14.wav", "F1_Ghain"),
    ("../Dataset/Zahrah_mic/Ghain_F2_14.wav", "F2_Ghain")
]

for folder in list_template_folders:
    clear_output()
    template_folder = os.path.join(root, folder)

    output_file = f"../fix_hasil_testing/non_ortho_prem_norm_testing_Ghain_template_{folder.strip('/')}.txt"

    template_score = 23.004
    max_threshold_score = 5

    with open(output_file, 'w') as f:
            f.write('')
            f.write(f"Banding {folder.strip('/')} dengan tester\n")
    print(f"Processing Folder = {folder.strip('/')}")
    # Iterate over each audio file
    for audio_input, audio_name in list_of_audio_files:
        main(audio_input,audio_name, template_folder, output_file, template_score, max_threshold_score)
    


Processing Folder = 10.Ha^'
rafi_Ghain

Total Distance = 334.0756745921165

Average Distance = 18.55975969956203

ridho_Ghain

Total Distance = 316.7619799799113

Average Distance = 17.59788777666174

F1_Ghain

Total Distance = 471.2288568352683

Average Distance = 26.179380935292684

F2_Ghain

Total Distance = 328.94231066620307

Average Distance = 18.27457281478906



In [21]:
# ini tes banding mfcc per audio langsung dtw

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from scipy.signal import butter, lfilter
from IPython.display import clear_output
from sklearn.decomposition import PCA

def remove_silence(y):
     # Memisahkan audio menjadi bagian-bagian berdasarkan energi
    parts = librosa.effects.split(y, top_db=25, frame_length=1024, hop_length=512)

    # Menggabungkan bagian-bagian yang tidak diam
    y_non_silent = []
    for start, end in parts:
        y_non_silent.extend(y[start:end])

    # Mengubah list menjadi array numpy
    y = np.array(y_non_silent)
    return y

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y = librosa.effects.preemphasis(y)
    y = y/np.max(np.abs(y))
    y = remove_silence(y)
    y = nr.reduce_noise(y, sr, prop_decrease=0.8)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    # y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=True,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm=None,
                                 dct_type=2,
                                 n_mels=20)
    # mfccs = mfccs[1:]
    # mfccs = librosa.util.normalize(mfccs, axis=1 )
    # return = librosa.util.normalize(mfccs.T)
    return mfccs.T


def apply_pca(mfcc_features, n_components):
    pca = PCA(n_components=n_components)
    pca.fit(mfcc_features)
    mfcc_pca = pca.transform(mfcc_features)
    return mfcc_pca

def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, audio_name, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    # print(f"len templates = {len(templates)}")
    total_distances = 0

    with open(output_file, 'a') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = np.loadtxt(audio_template,delimiter=',')
            # mfccs2 = mfccs2.T
            # mfccs2 = mfccs2[1:]
            # mfccs2 = mfccs2.T

            # mfccs1 = apply_pca(mfccs1,10)
            # mfccs2 = apply_pca(mfccs2,10)
            # print(f"mfccs1.shape = {mfccs1.shape}")
            # f.write(f"mfccs1.shape = {mfccs1.shape}")
            # print(f"mfccs2.shape = {mfccs2.shape}")
            # f.write(f"mfccs2.shape = {mfccs2.shape}")

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            # print(f"dtw_distance = {dtw_distance}")
            # f.write(f"dtw_distance = {dtw_distance}")
            total_distances += dtw_distance

            # result_line = f"Jarak DTW antara {audio_input} dan {template}: {dtw_distance}\n"
            # f.write(result_line)

        # Tulis hasil total jarak ke file
        f.write(audio_name)
        print(audio_name)
        f.write(f"\nTotal Distance = {total_distances}\n")
        print(f"\nTotal Distance = {total_distances}\n")

        # Hitung rata-rata jarak
        average_distance = total_distances / len(templates)
        f.write(f"Average Distance = {average_distance}\n")
        print(f"Average Distance = {average_distance}\n")
        # # Hitung selisih antara rata-rata jarak dengan nilai template
        # dif_distance = average_distance - template_score
        # f.write(f"Difference Distance = {dif_distance}\n")

        # # Hitung skor akhir berdasarkan selisih
        # if dif_distance <= 0:
        #     final_score = 100
        # elif dif_distance >= max_threshold_score:
        #     final_score = 0
        # else:
        #     final_score = int(100 - ((dif_distance / max_threshold_score) * 100))
        
        # f.write(f"Final Score = {final_score}\n")




root = "../Dataset_MFCC_prem_norm_non_ortho/"

list_template_folders = [
    ("01.Ha'/"),
    ("02.Kha'/"),
    ("03.Shad/"),
    ("04.Dhad/"),
    ("05.Tha'/"),
    ("06.Dhza'/"),
    ("07.'AIn/"),
    ("08.Ghain/"),
    ("09.Qaf/"),
    ("10.Ha^'/")
]
# List of audio files and their names
list_of_audio_files = [
    ("../Dataset/Testing_Rafi/rafi_qaf.wav", "rafi_qaf"),
    ("../Dataset/Ridho_mic/Ridho_qaf.wav", "ridho_qaf"),
    ("../Dataset/Dona_mic/qaf_F1_14.wav", "F1_qaf"),
    ("../Dataset/Zahrah_mic/qaf_F2_14.wav", "F2_qaf")
]

for folder in list_template_folders:
    clear_output()
    template_folder = os.path.join(root, folder)

    output_file = f"../fix_hasil_testing/non_ortho_prem_norm_testing_qaf_template_{folder.strip('/')}.txt"

    template_score = 23.004
    max_threshold_score = 5

    with open(output_file, 'w') as f:
            f.write('')
            f.write(f"Banding {folder.strip('/')} dengan tester\n")
    print(f"Processing Folder = {folder.strip('/')}")
    # Iterate over each audio file
    for audio_input, audio_name in list_of_audio_files:
        main(audio_input,audio_name, template_folder, output_file, template_score, max_threshold_score)
    


Processing Folder = 10.Ha^'
rafi_qaf

Total Distance = 293.883076583415

Average Distance = 16.3268375879675

ridho_qaf

Total Distance = 330.3730504702101

Average Distance = 18.35405835945612

F1_qaf

Total Distance = 415.3130290216632

Average Distance = 23.07294605675907

F2_qaf

Total Distance = 301.8847789716288

Average Distance = 16.771376609534933



In [1]:
# ini tes banding mfcc per audio langsung dtw

# sr = 44100
# n_fft = 1024 # 23ms = n_fft / sr
# hop_length = 512 # 10ms = hop_length / sr
import librosa
import noisereduce as nr
from fastdtw import fastdtw
from scipy.spatial.distance import cosine
import os
import numpy as np
# from sklearn.preprocessing import MaxAbsScaler
# from scipy.signal import butter, lfilter
from IPython.display import clear_output
# from sklearn.decomposition import PCA

def remove_silence(y):
     # Memisahkan audio menjadi bagian-bagian berdasarkan energi
    parts = librosa.effects.split(y, top_db=25, frame_length=1024, hop_length=512)

    # Menggabungkan bagian-bagian yang tidak diam
    y_non_silent = []
    for start, end in parts:
        y_non_silent.extend(y[start:end])

    # Mengubah list menjadi array numpy
    y = np.array(y_non_silent)
    return y

def preprocessing(audio):
    y , sr = librosa.load(audio, sr=44100)
    y = librosa.effects.preemphasis(y)
    y = y/np.max(np.abs(y))
    y = remove_silence(y)
    y = nr.reduce_noise(y, sr, prop_decrease=0.8)
    return y,sr

def extraction(audio):
    y , sr = preprocessing(audio)
    # y = librosa.effects.preemphasis(y)
    mfccs = librosa.feature.mfcc(y=y,
                                 sr=sr,
                                 n_mfcc=13,
                                 window='hamming',
                                 win_length=1024,
                                 htk=True,
                                 hop_length=512,
                                 n_fft=1024,
                                 norm=None,
                                 dct_type=2,
                                 n_mels=20)
    # mfccs = mfccs[1:]
    # mfccs = librosa.util.normalize(mfccs, axis=1 )
    # return = librosa.util.normalize(mfccs.T)
    return mfccs.T

def calculate_dtw(mfccs1, mfccs2):
    distance, _ = fastdtw(mfccs1, mfccs2, dist=cosine)
    return distance

def main(file_input, audio_name, folder_template, output_file, template_score, max_threshold_score):
    audio_input = file_input
    templates = os.listdir(folder_template)
    # print(f"len templates = {len(templates)}")
    total_distances = 0

    with open(output_file, 'a') as f:
        for template in templates:
            audio_template = os.path.join(folder_template, template)

            mfccs1 = extraction(audio_input)
            mfccs2 = np.loadtxt(audio_template,delimiter=',')
            # mfccs2 = mfccs2.T
            # mfccs2 = mfccs2[1:]
            # mfccs2 = mfccs2.T

            dtw_distance = calculate_dtw(mfccs1, mfccs2)
            # print(f"dtw_distance = {dtw_distance}")
            # f.write(f"dtw_distance = {dtw_distance}")
            total_distances += dtw_distance

            # result_line = f"Jarak DTW antara {audio_input} dan {template}: {dtw_distance}\n"
            # f.write(result_line)

        # Tulis hasil total jarak ke file
        f.write(audio_name)
        print(audio_name)
        f.write(f"\nTotal Distance = {total_distances}\n")
        print(f"\nTotal Distance = {total_distances}\n")

        # Hitung rata-rata jarak
        average_distance = total_distances / len(templates)
        f.write(f"Average Distance = {average_distance}\n")
        print(f"Average Distance = {average_distance}\n")
        # # Hitung selisih antara rata-rata jarak dengan nilai template
        # dif_distance = average_distance - template_score
        # f.write(f"Difference Distance = {dif_distance}\n")

        # # Hitung skor akhir berdasarkan selisih
        # if dif_distance <= 0:
        #     final_score = 100
        # elif dif_distance >= max_threshold_score:
        #     final_score = 0
        # else:
        #     final_score = int(100 - ((dif_distance / max_threshold_score) * 100))
        
        # f.write(f"Final Score = {final_score}\n")




root = "../Dataset_MFCC_prem_norm_non_ortho/"

list_template_folders = [
    # ("01.Ha'/"),
    # ("02.Kha'/"),
    # ("03.Shad/"),
    # ("04.Dhad/"),
    # ("05.Tha'/"),
    # ("06.Dhza'/"),
    # ("07.'AIn/"),
    # ("08.Ghain/"),
    # ("09.Qaf/"),
    ("10.Ha^'/")
]
# List of audio files and their names
list_of_audio_files = [
    ("../Dataset/Testing_Rafi/rafi_Ha^'.wav", "rafi_Ha^'"),
    # ("../Dataset/Ridho_mic/Ridho_Ha^'.wav", "ridho_Ha^'"),
    # ("../Dataset/Dona_mic/Ha^'_F1_14.wav", "F1_Ha^'"),
    # ("../Dataset/Zahrah_mic/Ha^'_F2_14.wav", "F2_Ha^'")
]

for folder in list_template_folders:
    clear_output()
    template_folder = os.path.join(root, folder)

    output_file = f"../fix_hasil_testing/tes.txt"

    template_score = 23.004
    max_threshold_score = 5

    with open(output_file, 'w') as f:
            f.write('')
            f.write(f"Banding {folder.strip('/')} dengan tester\n")
    print(f"Processing Folder = {folder.strip('/')}")
    # Iterate over each audio file
    for audio_input, audio_name in list_of_audio_files:
        main(audio_input,audio_name, template_folder, output_file, template_score, max_threshold_score)
    


Processing Folder = 10.Ha^'
rafi_Ha^'

Total Distance = 262.1023207615651

Average Distance = 14.561240042309173



In [9]:
import os
import numpy as np

root = "../Dataset_MFCC/"

list_template_folders = [
    "01.Ha'/",
    "02.Kha'/",
    "03.Shad/",
    "04.Dhad/",
    "05.Tha'/",
    "06.Dhza'/",
    "07.'AIn/",
    "08.Ghain/",
    "09.Qaf/",
    "10.Ha^'/"
]

for folderx in list_template_folders:
    foldernya = os.path.join(root, folderx)
    files = os.listdir(foldernya)
    for file in files:
        if file.endswith('.csv'):
            csv_filenya= np.loadtxt(os.path.join(foldernya, file), delimiter=',')
            print(f"Bentuk dari file {file}: {csv_filenya.shape}")


Bentuk dari file mfcc_Ha'_F1_11.csv: (783, 13)
Bentuk dari file mfcc_Ha'_F1_12.csv: (814, 13)
Bentuk dari file mfcc_Ha'_F1_13.csv: (823, 13)
Bentuk dari file mfcc_Ha'_F2_11.csv: (760, 13)
Bentuk dari file mfcc_Ha'_F2_12.csv: (732, 13)
Bentuk dari file mfcc_Ha'_F2_13.csv: (762, 13)
Bentuk dari file mfcc_Ha'_M1_11.csv: (812, 13)
Bentuk dari file mfcc_Ha'_M1_12.csv: (762, 13)
Bentuk dari file mfcc_Ha'_M1_13.csv: (784, 13)
Bentuk dari file mfcc_Ha'_M2_11.csv: (783, 13)
Bentuk dari file mfcc_Ha'_M2_12.csv: (695, 13)
Bentuk dari file mfcc_Ha'_M2_13.csv: (719, 13)
Bentuk dari file mfcc_Ha'_M3_1.csv: (1113, 13)
Bentuk dari file mfcc_Ha'_M3_2.csv: (1214, 13)
Bentuk dari file mfcc_Ha'_M3_3.csv: (1196, 13)
Bentuk dari file mfcc_Ha'_M4_1.csv: (1158, 13)
Bentuk dari file mfcc_Ha'_M4_2.csv: (1110, 13)
Bentuk dari file mfcc_Ha'_M4_3.csv: (1111, 13)
Bentuk dari file mfcc_Kha'_F1_11.csv: (830, 13)
Bentuk dari file mfcc_Kha'_F1_12.csv: (799, 13)
Bentuk dari file mfcc_Kha'_F1_13.csv: (797, 13)
Bentuk dar